In [1]:
import numpy as np
import pandas as pd
import gensim
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rosiezhao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
file = pd.read_csv('./data.csv')
file.head(10)

,id,title,author,date,content,year,month,publication,category,digital,section,url
0,1,Agent Cooper in Twin Peaks is the audience: on...,\nTasha Robinson\n,2017-05-31,And never more so than in Showtime’s new...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
1,2,"AI, the humanity!",\nSam Byford\n,2017-05-30,AlphaGo’s victory isn’t a defeat for hum...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
2,3,The Viral Machine,\nKaitlyn Tiffany\n,2017-05-25,Super Deluxe built a weird internet empi...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
3,4,How Anker is beating Apple and Samsung at thei...,\nNick Statt\n,2017-05-22,Steven Yang quit his job at Google in th...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
4,5,Tour Black Panther’s reimagined homeland with ...,\nKwame Opam\n,2017-05-15,Ahead of Black Panther’s 2018 theatrical...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
5,6,Instant recall,\nCasey Newton\n,2017-04-16,Facebook's Instant Articles promised to ...,2017.0,4.0,Verge,Longform,1.0,NaN,NaN
6,7,Massive attack,\nColin Lecher\n,2017-04-14,How a weapon against war became a weapon...,2017.0,4.0,Verge,Longform,1.0,NaN,NaN
7,8,The Future Agency,\nKyle Chayka\n,2017-03-30,Inside the big business of imagining the...,2017.0,3.0,Verge,Longform,1.0,NaN,NaN
8,9,Flight Risk,\nBen Popper\n,2017-03-28,Legal threats and disgruntled clients: i...,2017.0,3.0,Verge,Longform,1.0,NaN,NaN
9,10,Brain drain,\nCasey Newton\n,2017-03-15,Genius quietly laid off a bunch of its e...,2017.0,3.0,Verge,Longform,1.0,NaN,NaN


In [5]:
articles = file['content']



'''
Pre-processing:
>only retain words
>convert to lowercase
>remove stopwords
'''
articles = articles.str.replace('[^a-zA-Z]',' ').str.lower()
stop_re = '\\b'+'\\b|\\b'.join(nltk.corpus.stopwords.words('english'))+'\\b'
articles = articles.str.replace(stop_re, '')
articles = articles.str.split()

In [8]:
cleaned_articles = [x for x in articles if str(x) != 'nan']

In [13]:
#Input articles through Phraser
phrases = gensim.models.phrases.Phrases(cleaned_articles)
phraser = gensim.models.phrases.Phraser(phrases)
train_phrased = phraser[cleaned_articles]

In [14]:
#Create Word2Vec Model
w2v = gensim.models.word2vec.Word2Vec(sentences=train_phrased,workers=32)
print("Word2Vec Model finished training")

w2v.save('w2v_all-the-news')

Word2Vec Model finished training


/Users/rosiezhao/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [41]:
w2v.wv.most_similar('wildfire')

[('wildfires', 0.9039282202720642),
 ('blazes', 0.8697661757469177),
 ('floods', 0.830560564994812),
 ('mudslides', 0.8227199912071228),
 ('heavy_rains', 0.8194685578346252),
 ('floodwaters', 0.8102580904960632),
 ('severe_weather', 0.8025417327880859),
 ('torrential_rains', 0.8019133806228638),
 ('heavy_rain', 0.8009653091430664),
 ('tornado', 0.7995668649673462)]

In [40]:
w2v.wv.most_similar('flood')

[('flooding', 0.7642062902450562),
 ('deluge', 0.7069231867790222),
 ('floods', 0.689592719078064),
 ('flooded', 0.683785617351532),
 ('avalanche', 0.6525532007217407),
 ('tsunami', 0.6439600586891174),
 ('power_outages', 0.6375871896743774),
 ('mudslides', 0.6371573209762573),
 ('hurricane_harvey', 0.6213381290435791),
 ('storm', 0.610153079032898)]

In [38]:
w2v.wv.similarity('fire', 'flood')

0.43592682

In [ ]:
#Methods to calculate tf-idf scores
def word_count(text):
    '''
    Input:
    text - list object of tokenized strings
    '''
    return len(text)

def freq_count(articles):
    '''
    Input:
    articles - list of list of tokenized words
    Output:
    freq_dict_list - list of dictionary containing frequencies of each word in text
    '''
    freq_dict_list = []
    doc_id=0
    for text in articles:
        doc_id+=1
        freq_dict = {}
        for word in text:
            if word in freq_dict:
                freq_dict[word] += 1
            else:
                freq_dict[word] = 1
        doc_temp = {'id' : doc_id, 'freq_dict' : freq_dict }
        freq_dict_list.append(doc_temp)
    
    return freq_dict_list
    